<a href="https://colab.research.google.com/github/mosdef-hub/CECAM-MoSDeF-Workshop/blob/main/slitpore_workflow/Slitpore-Workflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Carbon Slitpore Workflow**
---
## Summary 
"Porous  carbon  materials  are  used  for  separation, purification, and catalysis purposes. While the adsorption and phase behavior of nonpolar fluids in carbon pores has  been  studied  extensively,our  understanding  regarding adsorption of water in carbonaceous materials is still rudimentary. Nevertheless, the structure and the thermodynamic  properties  of  water  confined  in  hydrophobic  regions  are  of  importance  in  many  scientific disciplines such as chemistry, geology, nanotechnology, and biology. Water adsorption in hydrophobic materials is typically characterized by negligible adsorption at low relative pressures, sudden and complete pore filling by a capillary-condensation mechanism, and large adsorption/ desorption hysteresis loops."

    - Striolo, A.; Chialvo, A. A.; Cummings, P. T.; Gubbins, K. E. Water Adsorption in Carbon-Slit Nanopores. Langmuir, 2003, 19 (20), 8583–8591.

The above study was recreated in 2020 in a work by Cummings et al. using open-source moleuclar modeling software with focus on the Molecular Simulation Design Framework (MoSDeF).

    - Peter Cummings, Clare McCabe, Christopher Iacovella, et al. Open-Source Molecular Modeling Software in Chemical Engineering Focusing on the Molecular Simulation Design Framework. Authorea. November 30, 2020.


## Learning Objectives
This notebook provides interactivre examples that will assist learners in using MoSDeF tools to:
- Create a molecule in different method using `mBuild`
- Load in a force field fromr XML and inspect the ForceField object with `GMSO`
- Parameterize a system with a force field and inspect the parameterized object
- Save out the topology and use it to run a Cassandra MC simulation using `mosdef_cassandra`

## Tutorial Contents
0. Set up environment on Google Colab
1. Construct System with mBuild
    1. Exercise 1a - Create a molecule with mbuild
    2. Exercise 1b - Pack a box of solvent
2. Load a ForceField
    1. Exercise 2 - Load and inspect a force field from XML
3. Parameterization
    1. Exercise 3 - Parameterize a compound/topology and summarize the parameterized object
4. Save out to Cassandra files
    1. Exercise 4 - Save a `.mcf` file from a typed Topology
5. Set up Cassandra input file and run simulation (optional)
## Software stack setup
After running the cell below the kernel will restart -- This is necessary for conda dependencies, but you'll need to wait for that kernel restart before running the second cell.


## Working with Google Colab
There are two types of output in these Colab notebooks that can be a little tricky:

1. If the output is very long, for example from the mamba command in the second cell, scrolling past the output can feel onerous. In this case, scrolling up and down in the narrow grey area between the sidebar menu and the cells can help you navigate.

2. If the output is a visualization of a molecule or simulation configuration, scrolling up or down will zoom in or out if the cursor is over the visualization. In these cases, take some care to scroll outside of the visualization.

3. To run a cell, either click the run button (right facing triangle) or hit `shift + enter`

## __0. Set up environment on Google Colab__
----

In [ ]:
# Note: Run this cell first and by itself. 
# The kernel will be restarted after this step 
# There might be an error pops up stating the session crashed
# for an unknown reason, but that is expected. 
!pip install -q condacolab
import condacolab
condacolab.install()

In [ ]:
import condacolab
condacolab.check()

!conda install mamba

!mamba install anaconda-client -n base
!git clone https://github.com/mosdef-hub/CECAM-MoSDeF-Workshop
!mamba env update -n base -f CECAM-MoSDeF-Workshop/environment.yml

## __1. Construct System with mBuild__
----
- The chemical system can be constructied with mBuild, the hierarchical molecular constructor of the MoSDeF software suite. The library offers several way to load or create a chemical systems, e.g., loading from common file format such as .xyz, .mol2, .pdb, from a SMILES string, using internal recipes, or user-construct recipes.
- Below, we demonstrate two methods of creating a molecule, i.e., using a SMILES string to create a water molecule, and using an user-recipe to build a carbon slitpore.

In [ ]:
%cd CECAM-MoSDeF-Workshop/slitpore_workflow

In [ ]:
import warnings 
warnings.filterwarnings("ignore")

# Import Libraries
import os
import mbuild as mb
import gmso
from porebuilder import GraphenePore

In [ ]:
# load molecules from their daylight SMILES string 
# https://www.daylight.com/dayhtml/doc/theory/theory.smiles.html
water = mb.load("O", smiles=True) 
water_box = mb.fill_box(water, box=[5,5,5], n_compounds=100) #density specify


"""Visualization utilities"""
water.print_hierarchy()

In [ ]:
water.visualize() # visualize molecule atoms and bonds

In [ ]:
# Load structure from recipes
graphene = GraphenePore(pore_length=4,
                        pore_depth=4,
                        n_sheets=1,
                        pore_width=1.2,
                        slit_pore_dim=1)
graphene.box = mb.Box([5, 5, 5])
# Try changing the n_sheets to form more layers
"""Visualization utility""" 
graphene.visualize()

### Exercise 1a - Create and visualize a system with mBuild (ET: 10 mins)
1. Create and visualize a molecule of choice with mbuild using SMILES string
    - Tips: Google molecule name + SMILES usually return the input you need
    - Note: you will need to set the options `smiles=True` in `mb.load()`
    - Tips: you can look at how we create the water molecule 2 cells
1. mBuid also supports loading a molecule/system from various file format
    - Download a pdb file from https://files.rcsb.org/view/1OIL.pdb using wget
    - Load in the file and visualize with `mbuild`

In [ ]:
# Exercise 1a.1
compound = mb.???(???, smiles=???)
compound.load()

In [ ]:
# Exercise 1a.2
!wget -O 1OIL.pdb https://files.rcsb.org/view/1OIL.pdb
protein = mb.???("1OIL.pdb")
protein.???

### <font color="red"><b>Exercise 1a Example Answer</b></font>

<details>
    <summary>Click once on to hide/unhide the answer!</summary>
    
        # Loading from a SMILES string 
         
        caffeine = mb.load("CN1C=NC2=C1C(=O)N(C(=O)N2C)C", smiles=True)
        caffeine.visualize()


        # Loading from a pdb file
    
    
        !wget -O 1OIL.pdb https://files.rcsb.org/view/1OIL.pdb
        protein = mb.load("1OIL.pdb")
        protein.visualize()
        
</details>

### Exercise 1b - Fill box and solvate (ET: 10 mins)
- mBuild utilize PACKMOL as the backend to perform packing molecules, solvating a solute. These functionalities are stored under `mbuild.packing`, with the two most frequently used methods being `packing.fill_box` and `packing.solvate`. Here, we will test out the `fill_box` method.
    - Create a packed box of ethanol following the procedure
        * Create an ethanol molecule using SMILES string
        * Pack a box of ethanol using the `packing.fill_box`, you will need to provide
            * `compound`: The molecule that is to be packed (expecting type `mb.Compound`)
            * `n_compounds`: The number of molecule (expecting type `int`)
            * `box`: The size of the box (define in form of [x, y, z], in all in nm)
        * Visualize the packed box

In [ ]:
# Exercise 1b.1
ethanol = mb.load(???, smiles=???)
ethanol_box = mb.???(compound=???,
                     n_compounds=???,
                     box=???
            )
ethanol_box.visualize()

### <font color="red"><b>Exercise 1b Example Answer</b></font>

<details>
    <summary>Click once on to hide/unhide the answer!</summary>
    
        ethanol = mb.load("CCO", smiles=True)
        ethanol_box = mb.fill_box(compound=ethanol,
                                  n_compounds=200,
                                  box=[3, 3, 3].
                    )
        ethanol_box.visualize()
</details>

## __2. Load A ForceField__
----

- In the MoSDeF ecosystem, we stored forcefield is stored in XML format, which contains information about version, combining rule, atom types, connection types and associated doi. Each atom type also includes a `def`, which stores the SMARTS definition, and `doi`, which store the original paper that the parameters are sourced from.
- Currently, there are two XML formats supported by MoSDeF tools, one of which is an extended version of OpenMM XML, while the other is newly developed to include more information that include additional information that we believe would be beneficial for performing TRUE research.

In [ ]:
carbon_forcefield = gmso.ForceField("../forcefields/carbon.xml")
carbon_forcefield

In [ ]:
"""Basic attributes of each atom type"""
for name, atype in carbon_forcefield.atom_types.items():
    print(atype)
    print("SMARTS definition:", atype.definition)
    print("Potential expression")
    display(atype.expression)
    print(atype.parameters)

In [ ]:
spce_forcefield = gmso.ForceField("../forcefields/spce.xml")
spce_forcefield

In [ ]:
"""Basic attributes of each connection type"""
for name, btype in spce_forcefield.bond_types.items():
    print(btype)
    print("Potential expression")
    display(btype.expression)
    print(btype.parameters)

### Exercise 2 - Load a force field and inspect some of its attributes (ET: 10mins)
1. Load the "OPLS" forcefield at `"../forcefields/oplsaa.xml"` to an object named `oplsaa` 
2. Inspect the forcefield 
    - Try calling `oplsaa.__dict__` and see all attributes that a force field has
    - What is the comining rule and scaling factor of this forcefield
3. Inspect some attributes of an atomtype 
    - Inspect the potential expression 
    - Notable attributes

In [ ]:
# Start your exercise here
oplsaa = gmso.???()

### <font color="red"><b>Exercise 2 Example Answer</b></font>

<details>
    <summary>Click once on to hide/unhide the answer!</summary>
    
        oplsaa = gmso.ForceField("../forcefields/oplsaa.xml")
        print(oplsaa.__dict__)
</details>

## __3. Parameterization__
----
- MoSDeF's backend data structure supports automatic atom typing and parameterization (mapping atom types and connection types stored in a loaded forcefield to a GMSO structure).
- This is done internally using Foyer, which performs graph matching between the molecule bond graph (of the GMSO Topology object) to the atom type SMARTS string. The algorithm for the processed is outlined in this [paper](https://www.journals.elsevier.com/computational-materials-science).
- The parameterization step created a typed Topology, which would be ready to be saved out to various file formats, ready to be taken in by corresponding simulation codes.

In [ ]:
from gmso.parameterization import apply

graphene_top = graphene.to_gmso()
single_water_top = water.to_gmso()
water_top = water_box.to_gmso()
graphene_ptop = apply(graphene_top, carbon_forcefield, identify_connections=True)
single_water_ptop = apply(single_water_top, spce_forcefield, identify_connections=True)
water_ptop = apply(water_top, spce_forcefield, identify_connections=True)

In [ ]:
# Iterable attributes
# graphene_top.sites
# graphene_top.bonds
# graphene_top.angles
# graphene_top.dihedrals
# graphene_top.impropers

display(graphene_ptop.sites[0].atom_type.expression)
print(f"{graphene_ptop.sites[0].atom_type.parameters}")

In [ ]:
"""Utility to output system as Dataframe"""
water_ptop.to_dataframe(site_attrs=["atom_type.parameters"])

In [ ]:
"""Utility to output system as Dataframe"""
graphene_ptop.to_dataframe(site_attrs=["atom_type.parameters"])

### Exercise 3 - Parametrized your solvent (ET: 10 mins)
1. Use the OPLS to try parameterize the molecule you created in the above exercise (it's may or may not be successful depends on how exotic the molecule you created)
    - Start by converting your compound to a GMSO `Topology`
    - Use the `apply` method to perform the parameterization.
    - Summarize the all the atomtypes in a dataframe 
2. Open the docstring for `Topology.to_dataframe`
    - See what you can modify the output of the dataframe to get the information you need.
    

In [ ]:
### Start your exercise here
topology = compound.to_gmso()
apply(???, 
      ???,
      identify_connection=True)

topology.???(site_attrs=["atom_type.parameters"])

### <font color="red"><b>Exercise 3 Example Answer</b></font>

<details>
    <summary>Click once on to hide/unhide the answer!</summary>
    
        # Parameterize the created compound with the OPLS-AA force field

        topology = compound.to_gmso()
        apply(topology, 
              oplsaa,
             identify_connection=True)

        topology.to_dataframe(site_attrs=["atom_type.parameters"])

        # Print out the docstring of Topology.to_dataframe
        help(Topology.to_dataframe) # Run this is a new cell

</details>

## __4. Save out to Cassandra files__
----
- The GMSO data structure provide direct support to multiple simulation engines, including GROMACS, LAMMPS, HOOMD-blue, GOMC and Cassandra. This includes the ability to directly save the typed Topology to molecular file input which can be used directly by the corresponding engines.
- In this example, we are writing out the file into Cassandra file format (`.mcf` or molecular connectivity file).

In [ ]:
# Saving out file and inspect the output
graphene_ptop.save("graphene.mcf", overwrite=True)
!cat graphene.mcf

### Exercise 4 - Save out the parameterized Water (ET: 5 mins)
Use similar syntax as above, save out the `.mcf` for the parameterized water (the `water_ptop` object created above) and print out the file (using `!cat`)
    

In [ ]:
### Start your exercise here
water_ptop.???(???, overwrite=???)
!cat ???

### <font color="red"><b>Exercise 4 Example Answer</b></font>

<details>
    <summary>Click once on to hide/unhide the answer!</summary>
    
        water_ptop.save("water.mcf", overwrite=True)
        !cat water.mcf

</details>

## __5. Set up Cassandra input file and run simulation (Optional)__
----
- In this step, we will attempt to use `mosdef_cassandra`, developed by Ryan DeFever et al., from the Maginn Group. The library provide utility to interface between MoSDeF core software stack to Cassandra. The library also provides a Python interface to define run-time parameters for the Monte Carlo simulation. Both `mosdef_cassandra` and `Cassandra` are installable through the `conda-forge` channel (for Linux and MacOS with Intel architecture).
- Here, we will run a short equilibration simulation just to demonstrate that we are able to write out are syntactically correct and are ready to be used as input for simulation engine. For the complete workflow, please refer to the original paper and its supplementa Github repository:
    - Peter Cummings, Clare McCabe, Christopher Iacovella, et al. Open-Source Molecular Modeling Software in Chemical Engineering Focusing on the Molecular Simulation Design Framework. Authorea. November 30, 2020.
    - https://github.com/mosdef-hub/mosdef_slitpore 

In [ ]:
import mosdef_cassandra as mc
from gmso.external import to_parmed
import unyt as u

# set variables
n_steps = 1000
temperature = 300 * u.K
mu = -54.0 * u.kJ / u.mol

# Create box and species list
box_list = [graphene]
species_list = [graphene_ptop.to_parmed(), 
                single_water_ptop.to_parmed()]

# Specify mols at start of the simulation
mols_in_boxes = [[1, 0]]

# Create MC system
system = mc.System(box_list, species_list, mols_in_boxes=mols_in_boxes)
moves = mc.MoveSet("gcmc", species_list)

# Set move probabilities
moves.prob_translate = 0.25
moves.prob_rotate = 0.25
moves.prob_insert = 0.25
moves.prob_regrow = 0.0

# Specify the restricted insertion
restricted_type = [[None, "slitpore"]]
restricted_value = [[None, 0.5 * 1.2 * u.nm]]
moves.add_restricted_insertions(
    species_list, restricted_type, restricted_value
)

# Set thermodynamic properties
thermo_props = [
    "energy_total",
    "energy_intervdw",
    "energy_interq",
    "nmols",
]

default_args = {
    "run_name" : "gcmc",
    "cutoff_style": "cut",
    "charge_style": "ewald",
    "rcut_min": 0.5 * u.angstrom,
    "vdw_cutoff": 9.0 * u.angstrom,
    "charge_cutoff": 9.0 * u.angstrom,
    "properties": thermo_props,
    "angle_style": ["harmonic", "fixed"],
    "coord_freq": 100000,
    "prop_freq": 1000,
}

custom_args = {**default_args}

mc.run(
    system=system,
    moveset=moves,
    run_type="equilibration",
    run_length=n_steps,
    temperature=temperature,
    chemical_potentials=["none", mu],
    **custom_args,
)